In [11]:
import pandas as pd
import polars as pl

from collections import Counter


pd.set_option('display.max_columns', None)

In [12]:
rawdata = pd.read_csv(
    "../data/fr.openfoodfacts.org.products.csv.gz",
    sep='\t',
    engine='python',
    nrows=1000000,  # Limite le nombre de lignes
    on_bad_lines='skip',  # Ignore les lignes mal formées
    quoting=3  # Pour gérer les guillemets mal échappés, cela correspond à CSV.QUOTE_NONE
)
french_data = rawdata[rawdata['countries_tags'] == 'en:france']
print(french_data.shape)

(124005, 206)


In [13]:
french_data = rawdata[rawdata['countries_tags'] == 'en:france']
print(french_data.shape)

(124005, 206)


In [14]:
# Find different states
unique_values = set()
french_data['states_tags'].str.split(',').apply(unique_values.update)

states_df = pd.DataFrame(data=unique_values, columns=['states'])
display(states_df)

,states
0,en:origins-completed
1,en:ingredients-photo-selected
2,en:nutrition-facts-completed
3,en:nutrition-photo-selected
4,en:photos-uploaded
5,en:packaging-photo-selected
6,en:packaging-photo-to-be-selected
7,en:ingredients-photo-to-be-selected
8,en:complete
9,en:expiration-date-to-be-completed


In [15]:
# Liste des états cibles
target_states = ["en:completed", "en:origins-completed", "en:ingredients-completed"]

# Construire une expression régulière pour les états
pattern = '|'.join(target_states)

# Compter les lignes contenant au moins un des états cibles
count = french_data['states_tags'].str.contains(pattern).sum()

print("Nombre de lignes avec les états spécifiés:", count)


Nombre de lignes avec les états spécifiés: 17671


In [16]:
columns_to_keep = [
    'code',
    'url',
    'created_datetime',
    'last_modified_datetime',
    'product_name',
    'packaging',
    'packaging_fr',
    'categories_tags',
    'categories_fr',
    'ingredients_text',
    'ingredients_tags',
    'ingredients_analysis_tags',
    'allergens',
    'traces_tags',
    'additives_n',
    'additives_tags',
    'nutriscore_score',
    'nutriscore_grade',
    'food_groups_tags',
    'states_tags',
    'ecoscore_score',
    'ecoscore_grade',
    'nutrient_levels_tags',
    'popularity_tags',
    'main_category',
    'image_url',
    'image_small_url',
    'image_ingredients_url',
    'image_ingredients_small_url',
    'image_nutrition_url',
    'image_nutrition_small_url',
    'energy-kcal_100g',
    'energy_100g',
    'fat_100g',
    'cholesterol_100g',
    'sugars_100g',
    'proteins_100g',
    'salt_100g',
    'alcohol_100g'
]

data = french_data[columns_to_keep]
display(data)

,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
4,00000005,http://world-fr.openfoodfacts.org/produit/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,Verre,"en:plant-based-foods-and-beverages,en:plant-ba...","Aliments et boissons à base de végétaux,Alimen...","Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,-2.0,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,208.0,840.0,0.0,NaN,8.0,0.0,0.23,NaN
9,00000010,http://world-fr.openfoodfacts.org/produit/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film e...","Plastique,Carton,Boite-en-carton,Film-en-plast...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","Snacks,Desserts,Snacks sucrés,Biscuits et gâte...","Farine de blé 33%, sucre, huile de colza, œufs...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",13.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.0,NaN,25.0,6.4,0.53,NaN
14,00000015,http://world-fr.openfoodfacts.org/produit/0000...,2018-04-15T16:43:14Z,2024-10-04T09:38:50Z,Madeleines ChocoLait,"Plastique, Carton","Plastique,Carton","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâtea...","Farine de blé 27%, chocolat au lait 18% (sucre...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",NaN,en:nuts,5.0,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",17.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",32.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,460.0,1926.0,24.0,NaN,31.0,6.4,0.48,0.0
18,00000020,http://world-fr.openfoodfacts.org/produit/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,1 boîte en carton à recycler 50 sachets indivi...,1-boite-en-carton-a-recycler-50-sachets-indivi...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés

In [17]:
# Basic stats
print("Taille du dataset:")
print("Number of rows : {}".format(data.shape[0]))
print("Number of columns : {}".format(data.shape[1]))
print()
print("---------------------------")
print()

print("Basics infos:")
print()
display(data.info())
print()
print("---------------------------")
print()

print("Basics statistics: ")
print()
data_desc = data.describe(include='all')
display(data_desc)
print()
print("---------------------------")
print()

print("Unique elements by feature: ")
print()
display(data.nunique().sort_values())
print()
print("---------------------------")
print()

print("Percentage of missing values: ")
print()
null_percentage = 100*data.isnull().mean()
null_percentage_df = null_percentage.to_frame(name='Null Percentage').T
display(null_percentage_df) 

Taille du dataset:
Number of rows : 124005
Number of columns : 39

---------------------------

Basics infos:

<class 'pandas.core.frame.DataFrame'>
Index: 124005 entries, 4 to 999998
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   code                         124005 non-null  object 
 1   url                          124005 non-null  object 
 2   created_datetime             124005 non-null  object 
 3   last_modified_datetime       124005 non-null  object 
 4   product_name                 115095 non-null  object 
 5   packaging                    10399 non-null   object 
 6   packaging_fr                 10399 non-null   object 
 7   categories_tags              47423 non-null   object 
 8   categories_fr                47423 non-null   object 
 9   ingredients_text             16553 non-null   object 
 10  ingredients_tags             16524 non-null   object 
 11  ingredients

None


---------------------------

Basics statistics: 



,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
count,124005,124005,124005,124005,115095,10399,10399,47423,47423,16553,16524,17790,4509,3067,16553.000000,7338,29696.000000,123096,41874,124005,35336.000000,123096,30845,22177,47423,108283,108283,40284,40284,57596,57596,7.442800e+04,7.525900e+04,74739.000000,406.000000,74614.000000,74773.000000,70065.000000,791.000000
unique,123946,123946,121427,105196,76494,1977,1652,7743,7743,13748,12938,35,387,763,NaN,3697,NaN,7,45,2418,NaN,9,191,5977,5136,108231,108231,40248,40248,57571,57571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,0202105021656,http://world-fr.openfoodfacts.org/produit/0202...,2021-09-21T15:30:45Z,2021-07-25T07:00:14Z,Filet de poulet,0,0,"en:meats-and-their-products,en:meats,en:chicke...","Viandes et dérivés,Viandes,Poulet et dérivés,V...",Bœuf,"en:chicken,en:poultry","en:palm-oil-content-unknown,en:vegan-status-un...",en:milk,en:nuts,NaN,"en:e322,en:e322i",NaN,unknown,"en:fish-meat-eggs,en:meat,en:poultry","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...","bottom-25-percent-scans-2020,bottom-20-percent...",en:chicken-breasts,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,2,7,51,627,1822,1086,2176,2176,295,240,4422,1023,357,NaN,327,NaN,92199,7221,16157,NaN,87305,2867,1832,2186,2,2,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.433154,NaN,8.890288,NaN,NaN,NaN,45.547204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.907884e+11,7.906031e+11,13.776127,0.025996,10.958449,11.395939,1.464180,2.780250
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.388427,NaN,8.816084,NaN,NaN,NaN,25.222706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.204993e+13,2.168891e+14,16.628091,0.053201,19.968930,14.697931,13.417748,6.694116
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,-15.000000,NaN,NaN,NaN,-25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,1.000000,NaN,NaN,NaN,25.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.300000e+02,5.400000e+02,1.900000,0.000000,0.500000,3.400000,0.150000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,11.000000,NaN,NaN,NaN,46.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.630000e+02,1.096000e+03,9.500000,0.000000,2.600000,8.000000,0.750000,0.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,16.000000,NaN,NaN,NaN,64.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.860000e+02,1.615000e+03,22.400000,0.030000,15.000000,19.000000,1.400000,4.050000



---------------------------

Unique elements by feature: 



nutriscore_grade                    7
ecoscore_grade                      9
additives_n                        22
ingredients_analysis_tags          35
food_groups_tags                   45
nutriscore_score                   53
alcohol_100g                       65
cholesterol_100g                  135
ecoscore_score                    144
nutrient_levels_tags              191
allergens                         387
traces_tags                       763
packaging_fr                     1652
packaging                        1977
states_tags                      2418
proteins_100g                    2698
energy_100g                      2754
fat_100g                         2952
sugars_100g                      3025
energy-kcal_100g                 3345
salt_100g                        3346
additives_tags                   3697
main_category                    5136
popularity_tags                  5977
categories_fr                    7743
categories_tags                  7743
ingredients_


---------------------------

Percentage of missing values: 



,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
Null Percentage,0.0,0.0,0.0,0.0,7.185194,91.614048,91.614048,61.757187,61.757187,86.651345,86.674731,85.653804,96.363856,97.526713,86.651345,94.082497,76.052579,0.733035,66.232007,0.0,71.504375,0.733035,75.126003,82.116044,61.757187,12.678521,12.678521,67.514213,67.514213,53.553486,53.553486,39.97984,39.309705,39.729043,99.672594,39.829846,39.701625,43.498246,99.362122


In [18]:
clean_data = data[data['ingredients_tags'].notnull()]
print(clean_data.shape)

(16524, 39)


In [19]:
file_path = '../data/fr.openfoodfacts.org.products.csv.gz'
chunk_size = 10000

filtered_chunks = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size, compression='gzip', sep='\t', engine='python', on_bad_lines='skip', quoting=3):
    filtered_chunk = chunk[(chunk['countries_tags'] == 'en:france') & (chunk['ingredients_tags'].notnull())]
    filtered_chunks.append(filtered_chunk)

if filtered_chunks:
    data_fr = pd.concat(filtered_chunks, axis=0)

data_fr = data_fr[columns_to_keep]
display(data_fr)
data_fr.to_csv('filtered_dataset_openfoodfacts_fr.csv')

,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
4,5,http://world-fr.openfoodfacts.org/produit/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,Verre,"en:plant-based-foods-and-beverages,en:plant-ba...","Aliments et boissons à base de végétaux,Alimen...","Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,-2.0,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,208.0,840.0,0.0,NaN,8.0,0.0,0.23,NaN
9,10,http://world-fr.openfoodfacts.org/produit/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film e...","Plastique,Carton,Boite-en-carton,Film-en-plast...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","Snacks,Desserts,Snacks sucrés,Biscuits et gâte...","Farine de blé 33%, sucre, huile de colza, œufs...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",13.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.0,NaN,25.0,6.4,0.53,NaN
14,15,http://world-fr.openfoodfacts.org/produit/0000...,2018-04-15T16:43:14Z,2024-10-04T09:38:50Z,Madeleines ChocoLait,"Plastique, Carton","Plastique,Carton","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâtea...","Farine de blé 27%, chocolat au lait 18% (sucre...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",NaN,en:nuts,5.0,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",17.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",32.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,460.0,1926.0,24.0,NaN,31.0,6.4,0.48,0.0
18,20,http://world-fr.openfoodfacts.org/produit/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,1 boîte en carton à recycler 50 sachets indivi...,1-boite-en-carton-a-recycler-50-sachets-indivi...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâte

In [20]:
filtered_chunk.to_csv('filtered_dataset_openfoodfacts.csv')

In [ ]:
file_path = '../data/en.openfoodfacts.org.products.csv.gz'
chunk_size = 10000

filtered_chunks_en = []

for chunk in pd.read_csv(file_path, chunksize=chunk_size, compression='gzip', sep='\t', engine='python', quoting=3):
    filtered_chunks_en = chunk[(chunk['countries_tags'] == 'en:france') & (chunk['ingredients_tags'].notnull())]
    filtered_chunks.append(filtered_chunks_en)

if filtered_chunks:
    data_en = pd.concat(filtered_chunks, axis=0)

data_en = data_en[columns_to_keep]
display(data_en)
data_en.to_csv('filtered_dataset_openfoodfacts_en.csv')

,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
4,5,http://world-fr.openfoodfacts.org/produit/0000...,2020-11-14T07:08:40Z,2024-10-20T14:02:34Z,Bio inulin,Glass,Verre,"en:plant-based-foods-and-beverages,en:plant-ba...","Aliments et boissons à base de végétaux,Alimen...","Jus et purée d'abricots (50%), eau, sucre.","en:apricot-juice-and-puree,en:fruit,en:prunus-...","en:palm-oil-free,en:vegan,en:vegetarian",NaN,NaN,0.0,NaN,-2.0,a,"en:fruits-and-vegetables,en:vegetables","en:to-be-completed,en:nutrition-facts-complete...",NaN,unknown,"en:fat-in-low-quantity,en:saturated-fat-in-low...",NaN,en:vegetables,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,208.0,840.0,0.0,NaN,8.0,0.0,0.23,NaN
9,10,http://world-fr.openfoodfacts.org/produit/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film e...","Plastique,Carton,Boite-en-carton,Film-en-plast...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","Snacks,Desserts,Snacks sucrés,Biscuits et gâte...","Farine de blé 33%, sucre, huile de colza, œufs...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",13.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.0,NaN,25.0,6.4,0.53,NaN
14,15,http://world-fr.openfoodfacts.org/produit/0000...,2018-04-15T16:43:14Z,2024-10-04T09:38:50Z,Madeleines ChocoLait,"Plastique, Carton","Plastique,Carton","en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâtea...","Farine de blé 27%, chocolat au lait 18% (sucre...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:maybe-vegetarian",NaN,en:nuts,5.0,"en:e322,en:e322i,en:e331,en:e422,en:e500,en:e503",17.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",32.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,460.0,1926.0,24.0,NaN,31.0,6.4,0.48,0.0
18,20,http://world-fr.openfoodfacts.org/produit/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,1 boîte en carton à recycler 50 sachets indivi...,1-boite-en-carton-a-recycler-50-sachets-indivi...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâte

In [22]:
data_en_allergen = data_en[data_en['allergens'].notna()]
data_en_allergen.shape

(203796, 39)

In [23]:
display(data_en_allergen)

,code,url,created_datetime,last_modified_datetime,product_name,packaging,packaging_fr,categories_tags,categories_fr,ingredients_text,ingredients_tags,ingredients_analysis_tags,allergens,traces_tags,additives_n,additives_tags,nutriscore_score,nutriscore_grade,food_groups_tags,states_tags,ecoscore_score,ecoscore_grade,nutrient_levels_tags,popularity_tags,main_category,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kcal_100g,energy_100g,fat_100g,cholesterol_100g,sugars_100g,proteins_100g,salt_100g,alcohol_100g
9,10,http://world-fr.openfoodfacts.org/produit/0000...,2016-10-20T07:19:01Z,2024-10-04T09:38:47Z,Madeleines nature,"Plastic,Cardboard,fr:Boîte en carton,fr:Film e...","Plastique,Carton,Boite-en-carton,Film-en-plast...","en:snacks,en:desserts,en:sweet-snacks,en:biscu...","Snacks,Desserts,Snacks sucrés,Biscuits et gâte...","Farine de blé 33%, sucre, huile de colza, œufs...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",3.0,"en:e331,en:e422,en:e503",13.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",53.0,c,"en:fat-in-high-quantity,en:saturated-fat-in-mo...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:plain-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,442.0,1852.0,22.00,NaN,25.00,6.4,0.5300,NaN
18,20,http://world-fr.openfoodfacts.org/produit/0000...,2018-09-14T13:14:06Z,2024-10-04T09:38:53Z,Madeleines Choco Noir,1 boîte en carton à recycler 50 sachets indivi...,1-boite-en-carton-a-recycler-50-sachets-indivi...,"en:snacks,en:sweet-snacks,en:biscuits-and-cake...","Snacks,Snacks sucrés,Biscuits et gâteaux,Gâtea...","Farine de _blé_ 27%, chocolat noir 18% (pâte d...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:palm-oil-free,en:non-vegan,en:vegetarian-st...","en:eggs,en:gluten,en:milk,en:soybeans",en:nuts,5.0,"en:e322,en:e331,en:e422,en:e500,en:e503",16.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-checked,en:complete,en:nutrition-fact...",39.0,d,"en:fat-in-high-quantity,en:saturated-fat-in-hi...","top-50000-scans-2019,top-100000-scans-2019,at-...",en:chocolate-madeleines,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,467.0,1953.0,25.00,NaN,29.00,6.3,0.4500,NaN
28,30,http://world-fr.openfoodfacts.org/produit/0000...,2016-12-15T22:15:44Z,2024-10-14T05:06:56Z,Confiture d'oranges,"Plastic,Bag,Box,Cardboard","Plastique,Carton,Sachet,Boîte","en:plant-based-foods-and-beverages,en:plant-ba...","Aliments et boissons à base de végétaux,Alimen...","Farine de _blé_, _œufs_ frais, huile de colza,...","en:wheat-flour,en:cereal,en:flour,en:wheat,en:...","en:may-contain-palm-oil,en:non-vegan,en:maybe-...","en:eggs,en:gluten,en:milk","en:nuts,en:soybeans",5.0,"en:e422,en:e450,en:e471,en:e500,en:e503",12.0,d,"en:sugary-snacks,en:biscuits-and-cakes","en:to-be-completed,en:nutrition-facts-complete...",76.0,a,"en:fat-in-moderate-quantity,en:saturated-fat-i...","bottom-25-percent-scans-2019,top-80-percent-sc...",fr:cakes-aux-raisins,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,https://images.openfoodfacts.org/images/produc...,0.0,1768.0,19.00,NaN,28.00,5.8,0.0000,NaN
48,50,http

In [24]:
unique_allergens = set()
data_en_allergen["allergens"].str.split(',').apply(unique_allergens.update)
allergens_list = sorted(unique_allergens)
print(len(allergens_list))
print(allergens_list)


1734
['ca:ordi', 'de:Dinkelweizenflocken', 'de:Dorschleber', 'de:ENTHÄLT EINE PHENYLALANINQUELLE', 'de:Eier und Eierzeugnisse.', 'de:KANN BEI ÜBERMÄSSIGEM VERZEHR ABFÜHREND WIRKEN', 'de:Kefir', 'de:Milcheiweißhydrolysat', 'de:Seehasen', 'de:Weizenstärkesirup', 'de:seigle', 'en:AVOINE', 'en:Almond flour', 'en:Amande', 'en:Amandes', 'en:Anhydride sulfureux et sulfites', 'en:Arachides', 'en:Arachnids', 'en:Avoine', 'en:BARLEY MALT', 'en:Barley malt', 'en:Beurre', 'en:Beurre salé', 'en:Blé', 'en:CREAMER', 'en:Cacahuètes', 'en:Cantal', 'en:Cereals Containing Gluten  (including Barley & Oats)', 'en:Crustacés', 'en:Crème', 'en:Céleri', 'en:E621', 'en:Emmental', 'en:Farine', 'en:Flétan', 'en:Fromage', 'en:Fruits à coque', 'en:Lacotse', 'en:Lacto', 'en:Lactosérum', 'en:Lait', "en:Malt d'orge", 'en:Malt de blé', 'en:Malted wheat', "en:Malts d'orge", 'en:Milch', 'en:Moutarde', 'en:Mozzarella', 'en:Noisette', 'en:Noisettes', 'en:Oat flakes', 'en:Oeufs', 'en:Orge', 'en:PHENYLALANINE', 'en:Phenylala

In [25]:
unique_allergens_tags = set()
data_en_allergen["traces_tags"].dropna().str.split(',').apply(unique_allergens_tags.update)
allergens_tags_list = sorted(unique_allergens_tags)
print(len(allergens_tags_list))
print(allergens_tags_list)


1251
['ca:api', 'de:andere-nusse', 'de:céleri', 'de:céréales-contenant-du-gluten-et-autres-fruits-à-coque-cacao-49-minimum-dans-le-chocolat-noir', 'de:céréales-contenant-du-gluten-et-fruits-à-coque', 'de:enthalt-möglicherwelse-gerste', 'de:erdnusse', 'de:fruits-à-coque', 'de:glucodies-3', 'de:gluten-et-autres-fruits-à-coque-cacao-30-minimum-dans-le-chocolat-au-lait', 'de:gluten-et-autres-fruits-à-coque-cacao-50-minimum-dans-le-chocolat-noir', 'de:gluten-et-autres-fruits-à-coques-cacao-50-minimum-dans-le-chocolat-noir', 'de:gluten-et-fruits-à-coque-cacao-39-minimum', 'de:gluten-toutes-nos-viandes-sont-issues-d-animaux-sacrifiés-selon-un-rite-islamique-et-contrôlées-par-un-organisme-indépendant-pourcentage-de-matières-grasses-inférieur-à-15-rapport-collagène-sur-protéines-de-viande-inférieur-à-25', 'de:graines-de-sésame', 'de:hasselnüsse', 'de:hühnereiweiß-milch-laktose', 'de:milch-und-milcherzeugnisse-einschließlich-lactose', 'de:nüsse-schalenteile', 'de:pekan', 'de:pinda-s', 'de:schale

In [26]:
# Compter les occurrences de chaque allergène
from collections import Counter

# Divise chaque entrée par ',' et compile tous les allergènes en une seule liste
allergens_list = data_en_allergen["allergens"].str.split(',').sum()

# Compte les occurrences de chaque allergène
allergen_counts = Counter(allergens_list)

# Convertir le résultat en DataFrame
allergens_df = pd.DataFrame(allergen_counts.items(), columns=['allergen', 'count']).sort_values(by='count', ascending=False)

print(allergens_df)

             allergen   count
2             en:milk  114032
1           en:gluten   89446
0             en:eggs   42228
3         en:soybeans   31256
5             en:nuts   21340
...               ...     ...
1725  en:WHEAT GLUTEN       2
1724       en:CREAMER       2
1723     fr:romatique       2
1722          fr:anit       2
1721          fr:E224       2

[1734 rows x 2 columns]
